In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

os.getcwd()

'/content'

In [ ]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/qbe/candidate_test")

In [ ]:
os.listdir()

['restaurants_train.csv',
 'restaurants_val.csv',
 'requirements.txt',
 'restaurants_holdout.csv',
 'README.pdf',
 '.~lock.restaurants_train.csv#',
 'entity_extraction.ipynb',
 '.ipynb_checkpoints',
 'rest_hold_preds_batchsize10.xlsx',
 'b_10_ss5',
 'b_10_cosine_sched',
 'bert_bs1',
 'b_1_ss50_e50',
 'b_1_ss50_e50_postag',
 'bert_bs2',
 'rest_hold_preds_batchsize10_20epochs.xlsx',
 'b_10_ss10_e50_swap']

In [ ]:
!pip install -r requirements.txt

In [ ]:
import pandas as pd

train = pd.read_csv('restaurants_train.csv')

val = pd.read_csv('restaurants_val.csv')

holdout = pd.read_csv('restaurants_holdout.csv')

train = train.fillna('')

val = val.fillna('')

holdout = holdout.fillna('')

In [ ]:
import random
def get_replacement_label(label, label_list):
    
    cands = [x for x in label_list if x != label]
    
    return cands[random.randint(0, len(cands)-1)]

def swap_labels(txt, label, replacement):
    
    return txt.replace(label, replacement)
    
train_swap = train.loc[train.restaurant_name != "",:].copy()

label_list = train_swap["restaurant_name"].unique().tolist()

train_swap["rep_label"] = train_swap["restaurant_name"].apply(lambda x: get_replacement_label(x, label_list))

train_swap["sentence"] = train_swap.apply(lambda x: swap_labels(x["sentence"], x["restaurant_name"], 
                                                                x["rep_label"]) , axis = 1)

train_swap["restaurant_name"] = train_swap["rep_label"]

train_swap

,sentence,restaurant_name,rep_label
4,can you find bellinis dedham pizzeria that hav...,bellinis,bellinis
6,can you tell me where the nearest bar la grass...,bar la grassa,bar la grassa
14,do you think thursday has fabulous service,thursday,thursday
17,can i get hambers at maid cafe,maid cafe,maid cafe
19,do you know if there are any reviews on bar la...,bar la grassa,bar la grassa
24,are there reservations still available for bel...,bellinis,bellinis
26,are there any olive garden in the city open on...,olive garden,olive garden
27,do i need a reservation for name,name,name
35,are there any name within 5 minutes drive that...,name,name
37,are there any dominos in town,dominos,dominos


In [ ]:
train = pd.concat([train, train_swap], axis = 0, ignore_index=True)

In [ ]:
train.shape

(160, 3)

In [ ]:
train = train.sample(frac = 1)

In [ ]:
import sys
import torch
import torch.nn as nn
import random

from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR
from transformers import BartForConditionalGeneration, BartTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

_ = model.to(device)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
def generate_outputs(model, sentence):
    inputs = tokenizer([sentence], max_length=1024, return_tensors='pt')
    output_ids = model.generate(inputs['input_ids'].to(device), num_beams=4)
    #print(output_ids, sentence)
    output = tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False).strip()
    return output

In [ ]:
train

,sentence,restaurant_name,rep_label
117,are there any chicken wing places nearby,,NaN
100,any good place to get a pie at an affordable p...,,NaN
29,anything on the avenue,,NaN
148,direct me to the nearest bar la grassa,bar la grassa,bar la grassa
63,can i dine at the barat a nossa casa,barat a nossa casa,NaN
...,...,...,...
31,are there any restaurants around with a smokin...,,NaN
111,are there any restaurants that are open 24 hours,,NaN
126,are there any olive garden in the city open on...,olive garden,olive garden
147,can you help me find a starting gate restauran...,starting gate restaurant,starting gate restaurant


In [ ]:
val

,sentence,restaurant_name
0,are there any ice cream shops in my neighborho...,
1,are there any restaurants within 5 miles that ...,
2,are there any locally owned franchises that gi...,
3,are there any restaurants that will let me tak...,
4,are there any five star restaurants around here,
5,do you think the noodle bar is open,noodle bar
6,are there any vegetarian restaurants in this town,
7,any places around here that has a nice view,
8,are there any jazz clubs that serve food,
9,do any famous people frequent the jimmys pizza...,jimmys pizza


In [ ]:
model.eval()
generate_outputs(model, train.sentence[6])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


'can you tell me where the nearest wendys is'

In [ ]:
model.train()
1

1

In [ ]:
################################
# DO NOT CHANGE THIS FUNCTION! #
################################

def get_f1_score_on_test_data(model, data):
    model.eval()
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    for index, row in data.iterrows():
        sentence = row.sentence
        expected = row.restaurant_name
        inputs = tokenizer([sentence], max_length=1024, return_tensors='pt')
        predicted = generate_outputs(model, sentence)
        if expected != '' and expected == predicted:
            true_positives += 1
        if expected != '' and expected != predicted:
            false_positives += 1
        if expected == '' and predicted != '':
            false_positives += 1
        if expected != '' and predicted == '':
            false_negatives += 1

    precision = 0
    recall = 0
    f1_score = 0
    if true_positives + false_positives:
        precision = true_positives / (true_positives + false_positives)
    if true_positives + false_negatives:
        recall = true_positives /(true_positives + false_negatives)
    if precision + recall:
        f1_score = 2 * precision * recall / (precision + recall)

    print(f'precision: {precision} | recall {recall} | f1_score {f1_score}')
    return f1_score

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

_ = model.to(device)

num_epochs = 20
learning_rate = 2e-5
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=10, gamma=0.8)

In [ ]:
# from torch.nn.utils.rnn import pad_sequence

In [ ]:
# tokenizer.batch_encode_plus?

In [ ]:
# train.sentence[0]

In [ ]:
# tokenizer.batch_encode_plus([train.sentence[0]])["input_ids"]

In [ ]:
# tokenizer.encode?
# #(train.sentence[0], return_tensors='pt')

In [ ]:
# sents = [x.sentence for row, x in train.iterrows()]

# labels = [x.restaurant_name for row, x in train.iterrows()]

# start = 0

# tokenizer.batch_encode_plus(sents[start:start+10], padding=True)["input_ids"]

In [ ]:
def train_one_batch_and_get_loss(train_model, data, optimizer, criterion, batch_size=10):
    total_loss = 0.

    sents = [x.sentence for row, x in data.iterrows()]
    labels = [x.restaurant_name for row, x in data.iterrows()]

    #sents_tok = [tokenizer.encode(x, padding=True) for x in sents]

    #labels_tok = [tokenizer.encode(x, padding=True) for x in labels]

    total_steps = int(len(sents) / batch_size)

    for i in tqdm(range(total_steps), total=total_steps): 
        start = i * batch_size       
        #print(start, start+batch_size)
        batch_sents = torch.tensor(tokenizer.batch_encode_plus(sents[start:start+batch_size], padding=True)["input_ids"]).to(device)
        
        batch_labels = torch.tensor(tokenizer.batch_encode_plus(labels[start:start+batch_size], padding=True)["input_ids"]).to(device)
        model.train()
        optimizer.zero_grad()
        
        loss = train_model(batch_sents, labels=batch_labels)[0]
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / data.shape[0]

In [ ]:
def train_one_step_and_get_loss(train_model, data, optimizer, criterion):
    total_loss = 0.
    for i, row in tqdm(data.iterrows(), total=data.shape[0]):
        model.train()
        optimizer.zero_grad()
        input_ids = tokenizer.encode(row.sentence, return_tensors='pt').to(device)
        output_ids = tokenizer.encode(row.restaurant_name, return_tensors='pt').to(device)
        loss = train_model(input_ids, labels=output_ids)[0]
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / data.shape[0]

In [ ]:
best_model = None
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

_ = model.to(device)

num_epochs = 50
learning_rate = 2e-5
batch_size = 10
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=10, gamma=0.8)
min_f1 = 0
holdout_f1 = 0
MODEL_PATH = "b_10_ss10_e50_swap"
for epoch in range(num_epochs):
    loss = train_one_batch_and_get_loss(model, train, optimizer, criterion, batch_size=batch_size)
    print('Epoch:', epoch , '-' * 35)
    print('Training loss:', loss)
    f1_val = get_f1_score_on_test_data(model, val)
    f1_train = get_f1_score_on_test_data(model, train)
    f1 = get_f1_score_on_test_data(model, holdout)
    if f1_val > min_f1:
        
        print("saving model:", f1, f1_val,min_f1, epoch)
        min_f1 = f1_val
        holdout_f1 = f1
        torch.save(model.state_dict(), MODEL_PATH)
    best_model = model
    print('-' * 44)
    sys.stdout.flush()
    scheduler.step()

100%|██████████| 16/16 [00:03<00:00,  4.52it/s]
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Epoch: 0 -----------------------------------
Training loss: 0.7854907304048538
precision: 0.0 | recall 0.0 | f1_score 0
precision: 0.012048192771084338 | recall 0.013888888888888888 | f1_score 0.012903225806451613
precision: 0.0 | recall 0.0 | f1_score 0
--------------------------------------------


100%|██████████| 16/16 [00:03<00:00,  4.70it/s]


Epoch: 1 -----------------------------------
Training loss: 0.5994654268026351
precision: 0.25 | recall 0.375 | f1_score 0.3
precision: 0.4 | recall 0.6181818181818182 | f1_score 0.48571428571428577
precision: 0.14285714285714285 | recall 0.25 | f1_score 0.18181818181818182
saving model: 0.18181818181818182 0.3 0 1
--------------------------------------------


100%|██████████| 16/16 [00:03<00:00,  4.53it/s]


...

Epoch: 47 -----------------------------------
Training loss: 0.0016032239422202111
precision: 0.5 | recall 0.75 | f1_score 0.6
precision: 0.9512195121951219 | recall 1.0 | f1_score 0.975
precision: 0.28421052631578947 | recall 0.8181818181818182 | f1_score 0.42187499999999994
--------------------------------------------


100%|██████████| 16/16 [00:03<00:00,  4.68it/s]


Epoch: 48 -----------------------------------
Training loss: 0.0019000217493157835
precision: 0.5833333333333334 | recall 0.875 | f1_score 0.7000000000000001
precision: 0.9512195121951219 | recall 1.0 | f1_score 0.975
precision: 0.3 | recall 0.7297297297297297 | f1_score 0.4251968503937008
--------------------------------------------


100%|██████████| 16/16 [00:03<00:00,  4.69it/s]


Epoch: 49 -----------------------------------
Training loss: 0.003466495400061831
precision: 0.5833333333333334 | recall 0.875 | f1_score 0.7000000000000001
precision: 0.9634146341463414 | recall 1.0 | f1_score 0.9813664596273292
precision: 0.29347826086956524 | recall 0.7941176470588235 | f1_score 0.42857142857142855
--------------------------------------------


In [ ]:
holdout_f1

0.425531914893617

In [ ]:
best_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

best_model.load_state_dict(torch.load(MODEL_PATH))

best_model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        

In [ ]:
get_f1_score_on_test_data(best_model, holdout)

precision: 0.30612244897959184 | recall 0.6976744186046512 | f1_score 0.425531914893617


0.425531914893617

In [ ]:
get_f1_score_on_test_data(model, holdout)

precision: 0.29347826086956524 | recall 0.7941176470588235 | f1_score 0.42857142857142855


0.42857142857142855

In [ ]:
model.eval()
holdout["preds"] = holdout["sentence"].apply(lambda x: generate_outputs(model, x))

holdout

,sentence,restaurant_name,preds
0,find pizza places,,
1,find me the best rated chinese restaurant in t...,,chinese restaurant
2,what kind of food does abc cafe serve,abc cafe,abc cafe
3,how far away is the nearest steak house,,
4,i am looking for a mexican restuarant that has...,,
...,...,...,...
145,find me brazilian food with on location parking,,
146,get me to a mexican place,,
147,how far am i from the nearest bagel shop,,bagel shop
148,what time does sonic open,sonic,dominos


In [ ]:
def generate_outputs(model, sentence):
    inputs = tokenizer([sentence], max_length=1024, return_tensors='pt')
    output_ids = model.generate(inputs['input_ids'].to(device), num_beams=4)
    #print(output_ids, sentence)
    output = tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False).strip()

    if output in sentence:
        return output
    else:
        return ""

In [ ]:
get_f1_score_on_test_data(model, holdout)

precision: 0.32142857142857145 | recall 0.5510204081632653 | f1_score 0.40601503759398494


0.40601503759398494

In [ ]:
holdout["preds2"] = holdout["sentence"].apply(lambda x: generate_outputs(model, x))

holdout

,sentence,restaurant_name,preds,preds2
0,find pizza places,,,
1,find me the best rated chinese restaurant in t...,,chinese restaurant,chinese restaurant
2,what kind of food does abc cafe serve,abc cafe,abc cafe,abc cafe
3,how far away is the nearest steak house,,,
4,i am looking for a mexican restuarant that has...,,,
...,...,...,...,...
145,find me brazilian food with on location parking,,,
146,get me to a mexican place,,,
147,how far am i from the nearest bagel shop,,bagel shop,bagel shop
148,what time does sonic open,sonic,dominos,


In [ ]:
holdout.to_excel("rest_hold_preds_batchsize10_20epochs.xlsx", index=False)

In [ ]:
num = 0
print('input:', holdout.sentence[num])
print('expected:', holdout.restaurant_name[num])
print('predicted:', generate_outputs(best_model, holdout.sentence[num]))

input: find pizza places
expected: 
predicted: pizza places
